In [170]:
import numpy as np
import random
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

In [171]:
# load the dataset into train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [172]:
print("train data")
print(x_train.shape)
print(y_train.shape)
print("\n test data")
print(x_test.shape)
print(y_test.shape)

train data
(60000, 28, 28)
(60000,)

 test data
(10000, 28, 28)
(10000,)


In [173]:
# sample only 20k images for training
idx = np.random.randint(x_train.shape[0], size=20000) # sample 20k indices from 0-60,000
x_train = x_train[idx, :]
y_train = y_train[idx]
print(x_train.shape)
print(y_train.shape)

(20000, 28, 28)
(20000,)


In [174]:
# specify input dimensions of each image
img_rows, img_cols = 28, 28
input_shape = (img_rows, img_cols, 1)

# batch size, number of classes, epochs
batch_size = 128
num_classes = 10
epochs = 12

In [175]:
# reshape x_train and x_test
x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
print(x_train.shape)
print(x_test.shape)

(20000, 28, 28, 1)
(10000, 28, 28, 1)


In [176]:
# convert class labels (from digits) to one-hot encoded vectors
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
print(y_train.shape)

(20000, 10)


In [177]:
# originally, the pixels are stored as ints
x_train.dtype

dtype('uint8')

In [178]:
# convert int to float
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

# normalise
x_train /= 255
x_test /= 255

In [179]:
# model
model = Sequential()

# a keras convolutional layer is called Conv2D
# help(Conv2D)
# note that the first layer needs to be told the input shape explicitly

# first conv layer
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape)) # input shape = (img_rows, img_cols, 1)

# second conv layer
model.add(Conv2D(64, kernel_size=(3, 3), 
                 activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# flatten and put a fully connected layer
model.add(Flatten())
model.add(Dense(128, activation='relu')) # fully connected
model.add(Dropout(0.5))

# softmax layer
model.add(Dense(num_classes, activation='softmax'))

# model summary
model.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_22 (Conv2D)           (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_23 (Conv2D)           (None, 24, 24, 64)        18496     
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 12, 12, 64)        0         
_________________________________________________________________
dropout_21 (Dropout)         (None, 12, 12, 64)        0         
_________________________________________________________________
flatten_11 (Flatten)         (None, 9216)              0         
_________________________________________________________________
dense_21 (Dense)             (None, 128)               1179776   
_________________________________________________________________
dropout_22 (Dropout)         (None, 128)               0         
__________

In [180]:
# usual cross entropy loss
# choose any optimiser such as adam, rmsprop etc
# metric is accuracy
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

In [181]:
# fit the model
# this should take around 10-15 minutes when run locally on a windows/mac PC 
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))

Train on 20000 samples, validate on 10000 samples
Epoch 1/12
20000/20000 [==============================] - 62s 3ms/step - loss: 0.4833 - acc: 0.8492 - val_loss: 0.1201 - val_acc: 0.9625
Epoch 2/12
20000/20000 [==============================] - 63s 3ms/step - loss: 0.1486 - acc: 0.9558 - val_loss: 0.0805 - val_acc: 0.9749
Epoch 3/12
20000/20000 [==============================] - 61s 3ms/step - loss: 0.1057 - acc: 0.9683 - val_loss: 0.0659 - val_acc: 0.9790
Epoch 4/12
20000/20000 [==============================] - 61s 3ms/step - loss: 0.0795 - acc: 0.9750 - val_loss: 0.0673 - val_acc: 0.9769
Epoch 5/12
20000/20000 [==============================] - 61s 3ms/step - loss: 0.0631 - acc: 0.9800 - val_loss: 0.0534 - val_acc: 0.9828
Epoch 6/12
20000/20000 [==============================] - 63s 3ms/step - loss: 0.0509 - acc: 0.9838 - val_loss: 0.0513 - val_acc: 0.9835
Epoch 7/12
20000/20000 [==============================] - 63s 3ms/step - loss: 0.0475 - acc: 0.9843 - val_loss: 0.0453 - val_acc

In [193]:
# evaluate the model on test data
model.evaluate(x_test, y_test)

10000/10000 [==============================] - 9s 878us/step


[0.045219762670705134, 0.9859]

In [194]:
print(model.metrics_names)

['loss', 'acc']


The final loss (on test data) is about 0.04 and the accuracy is 98.59%.